In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# from keras import Sequential
# import keras
# from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Reshape, InputLayer, Dropout
# import tensorflow as tf

%matplotlib inline


def read_bed(filename):
    return pd.read_csv(filename, sep='\t', header=None)


def to_bed(data, filename):
    data.to_csv(filename, sep='\t', header=None, index=None)


N = 100
bps = 100
chr_names = list(read_bed('plus.bed')[0].unique())
histones = ['E003-H3K4me1', 'E003-H3K4me3', 'E003-H3K9me3', 'E003-H3K27me3', 'E003-H3K36me3']

In [3]:
print(chr_names)
print(histones)

['chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr1', 'chr20', 'chr21', 'chr22', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrX', 'chrY', 'chrM']
['E003-H3K4me1', 'E003-H3K4me3', 'E003-H3K9me3', 'E003-H3K27me3', 'E003-H3K36me3']


In [25]:
# regions

,0,1,2
0,chr10,85139,95139
1,chr10,85627,95627
2,chr10,88064,98064
3,chr10,88927,98927
4,chr10,90030,100030
...,...,...,...
17001,chr10,135482557,135492557
17002,chr10,135485250,135495250
17003,chr10,135493820,135503820
17004,chr10,135493970,135503970


In [17]:
def regionize(regions, N, bps):
    regions_100 = pd.DataFrame(
        np.zeros((regions.shape[0] * N, 3), dtype=np.int64),
    )
    for i in range(N):
        regions_100.iloc[i::N, 0] = np.array(regions[0])
        regions_100.iloc[i::N, 1] = np.array(regions[1]) + i * bps
        regions_100.iloc[i::N, 2] = np.array(regions[2]) + (i + 1) * bps
    return regions_100

In [49]:
print('size:', N * bps)

regions_raw = read_bed('plus.bed')
for chr_name in ['chr10']:
    print(chr_name)
    regions = regions_raw[regions_raw[0] == chr_name].copy()
    regions.index = range(regions.shape[0])
    regions[1], regions[2] = (regions[1] + regions[2]) // 2 -  N * bps // 2, (regions[1] + regions[2]) // 2 + N * bps // 2
    regions
    print('\t +1')
    regions_random = pd.DataFrame(
        np.zeros((regions.shape[0] - 1, regions.shape[1]), dtype=np.int64),
    )
    regions_random[0] = regions[0][0]

    regions_random[1] = np.array(regions.iloc[:-1, 2]) # minimum start
    regions_random[2] = np.array(regions.iloc[1:, 1]) # maximum end
    regions_random[3] = regions_random[2] - regions_random[1]
    print((regions_random[3] > 100000).sum())
    
#     regions_random = regions_random[regions_random[3] >= N * bps]
#     regions_random[1], regions_random[2] = (regions_random[1] + regions_random[2]) // 2 -  N * bps // 2, (regions_random[1] + regions_random[2]) // 2 + N * bps // 2
    
#     print('\t +0')
#     with open('{}k/{}_regions_num.txt'.format(N * bps // 1000, chr_name), 'w+') as file_num:
#         print(regions.shape[0], regions_random.shape[0], file=file_num)
#     print('\t n', regions.shape[0], regions_random.shape[0])

#     to_bed(regionize(regions, N, bps), '{}k/plus_{}.bed'.format(N * bps // 1000, chr_name))
#     print('\t 1')

#     to_bed(regionize(regions_random, N, bps), '{}k/plus_{}_random.bed'.format(N * bps // 1000, chr_name))
#     print('\t 0')

size: 5000
chr10
	 +1
21


In [63]:
regions_random = pd.DataFrame(
    np.zeros((regions.shape[0] - 1, regions.shape[1]), dtype=np.int64),
)
regions_random[0] = regions[0][0]

regions_random[1] = np.array(regions.iloc[:-1, 2]) # minimum start
regions_random[2] = np.array(regions.iloc[1:, 1]) # maximum end
regions_random[3] = regions_random[2] - regions_random[1]
# print((regions_random[3] > 100000).sum())
regions_random = regions_random[regions_random[3] >= N * bps]

regions_random_list = []
for i in range(regions_random.shape[0]):
    d = 1
    while d * N * bps <= regions_random.iloc[i, 3]:
        regions_random_list.append((regions_random.iloc[i, 1] + (d - 1) * N * bps, regions_random.iloc[i, 1] + (d) * N * bps))
        d += 1

len(regions_random_list)
regions_random = pd.DataFrame(
    np.zeros((len(regions_random_list), 3), dtype=np.int64)
)
for i in range(regions_random.shape[0]):
    regions_random.iloc[i, :] = [chr_name, *regions_random_list[i]]

regions_random

,0,1,2
0,chr10,97755,107755
1,chr10,133626,143626
2,chr10,167559,177559
3,chr10,183601,193601
4,chr10,193601,203601
...,...,...,...
5669,chr10,135317517,135327517
5670,chr10,135327517,135337517
5671,chr10,135346510,135356510
5672,chr10,135408603,135418603


In [64]:
regionize(regions_random, N, bps)

,0,1,2
0,chr10,97755,107855
1,chr10,97855,107955
2,chr10,97955,108055
3,chr10,98055,108155
4,chr10,98155,108255
...,...,...,...
567395,chr10,135510970,135521070
567396,chr10,135511070,135521170
567397,chr10,135511170,135521270
567398,chr10,135511270,135521370


In [65]:
to_bed(regionize(regions_random, N, bps), '{}k/plus_{}_random_many.bed'.format(N * bps // 1000, chr_name))

In [66]:
read_bed('10k/plus_chr10_random_many.bed')

,0,1,2
0,chr10,97755,107855
1,chr10,97855,107955
2,chr10,97955,108055
3,chr10,98055,108155
4,chr10,98155,108255
...,...,...,...
567395,chr10,135510970,135521070
567396,chr10,135511070,135521170
567397,chr10,135511170,135521270
567398,chr10,135511270,135521370
